In [1]:
from naqst import *
from openpyxl import Workbook
import math
import time
from vfsexp import Vf
from Enola.codegen import CodeGen, global_dict
from Enola.route import QuantumRouter
import json
from qiskit.qasm2.export import dump

In [25]:
from qiskit import QuantumCircuit
from qiskit import transpile

cirr = Qu

In [28]:
cir = CreateCircuitFromQASM('qft_5.qasm', 'Data/qft/')
cz_circuit = transpile(cir, basis_gates=['cz', 'rx', 'ry', 'rz', 'h', 't'])
print(cz_circuit)
dump(cz_circuit, 'cz_qft_5.qasm')

Data/qft/qft_5.qasm
global phase: 0.83449
        ┌───┐                                                            »
q_0: ───┤ H ├────────────────────────────────────────────────────────────»
        ├───┤                                                            »
q_1: ───┤ H ├────────────────────────────────────────────────────────────»
        ├───┤                                                            »
q_2: ───┤ H ├──────────────────────────────────────────────────────────■─»
        ├───┤                    ┌──────────┐   ┌─────────┐┌─────────┐ │ »
q_3: ───┤ H ├──────────────────■─┤ Rx(-π/4) ├─■─┤ Rx(π/4) ├┤ Rz(π/4) ├─┼─»
     ┌──┴───┴───┐┌───────────┐ │ └──────────┘ │ ├─────────┤└─────────┘ │ »
q_4: ┤ Ry(-π/2) ├┤ Rz(-3π/4) ├─■──────────────■─┤ Rz(π/8) ├────────────■─»
     └──────────┘└───────────┘                  └─────────┘              »
«                                                                     »
«q_0: ───────────────────────────────────────────────────────